In [3]:
from sklearn.metrics import r2_score



In [1]:
import pickle
import tensorflow as tf
import numpy as np

from keras.layers import Lambda,Reshape,concatenate,Input, Embedding, LSTM,GRU
from keras.layers import Dense,Dropout, Activation ,Flatten ,RepeatVector, Bidirectional,GlobalAveragePooling1D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.activations import softmax
from keras import regularizers

from keras import backend as K, regularizers, constraints, initializers
from keras.engine.topology import Layer

from keras.layers import merge
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.models import Sequential

from keras.callbacks import EarlyStopping


from Attention import Attention

from keras.layers import Concatenate,Dot
from keras.layers import Permute, merge

# FOR ATAE
from AttentionwithContext import AttentionWithContext
from Final import FinalSentenceRepresentation

from final2 import Final2

from keras.models import model_from_json

from scipy.interpolate import interp1d

import keras

import os
import pickle
import codecs
import nltk
from gensim.models import Word2Vec

Using TensorFlow backend.
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np
import json
import pandas as pd
from scipy import interpolate
from gensim.models import KeyedVectors
import pickle
import re

from nltk.corpus import stopwords
from nltk import word_tokenize

from nltk import pos_tag
from string import punctuation,digits
import os
import pickle

from sklearn import preprocessing

import tensorflow as tf

from scipy.interpolate import interp1d

from keras.utils import to_categorical 


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
from keras.models import model_from_json
from sklearn.metrics import f1_score,accuracy_score


In [7]:
from gensim.models import FastText
import gensim

In [8]:
train_data=pickle.load(open(r"D:\PythonCodes\Sentiment-Analysis\Code\train_data_initial.dat","rb"))

In [9]:
head_data=pickle.load(open('test_head.dat',"rb"))
post_data=pickle.load(open('test_post.dat',"rb"))
dataframe=pickle.load(open('dataframe.dat',"rb"))

In [10]:
def sk_mse(y_true,y_pred):
     return np.mean(np.square(y_pred - y_true), axis=-1)

a=pickle.load(open("D:/PythonCodes/Sentiment-Analysis/Code/ALLdataForSentiment.dat","rb"))

a.keys()

trainX=a['trainX']
trainY=a['trainY']
embedding_matrix=a['embedding_matrix']
head_X=a['HEAD_testX']
head_Y=a['HEAD_testY']
post_Y=a['POST_testY']
post_X=a['POST_testX']
target = a['target']
def sk_mse(y_true,y_pred):
     return np.mean(np.square(y_pred - y_true), axis=-1)


max_length=15

vocab_size=a['vocab_size']


# TAKING ONLY POST DATA

In [11]:
post=pickle.load(open('post.dat',"rb"))
post.head()

,aspect,features,sentence,sentiment,snippet
0,Price Action,[],COMPANY big money pouring facebook directions,0.645,big money is pouring into
1,Price Action,[],COMPANY guess want look sell orders,-0.465,guess they want it down look at the those sell...
2,Technical Analysis,[],COMPANY bounced support early week indicators ...,0.351,"now indicators turning up (rsi, macd, smi)"
3,Price Action,[],OCOMPANY OCOMPANY COMPANY long morning,0.476,long this morning
4,Price Action,[2],COMPANY pos saying years going lower forget go...,-0.631,going lower. forget about it or go short.


In [12]:
pickle.dump(post,open('post.dat',"wb"))

In [13]:
pickle.dump(post_data,open('test_post.dat',"wb"))

# Tokenizing

In [14]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [15]:
import random
c = list(zip(train_data['sentiment'], train_data['sentence'],train_data['aspect']))
random.shuffle(c)
train_data['sentiment'], train_data['sentence'],train_data['aspect']= zip(*c)

In [16]:
train_data['sentiment']=list(train_data['sentiment'])
train_data['sentence']=list(train_data['sentence'])
train_data['aspect']=list(train_data['aspect'])

In [17]:
val_test=[]
val_y=[]
for i in post_data['snippets']:
    val_test.append(i)
for i in head_data['snippets']:
    val_test.append(i)
for i in post_data['sentiment']:
    val_y.append(i)
for i in head_data['sentiment']:
    val_y.append(i)

In [18]:

c = list(zip(val_test, val_y))
random.shuffle(c)
val_test, val_y= zip(*c)

In [19]:
val_test=list(val_test)
val_y=list(val_y)

In [20]:
all_data=[]
for i in dataframe['sentence']:
    all_data.append(i)
for i in head_data['sentence']:
    all_data.append(i)
for i in post_data['sentence']:
    all_data.append(i)

In [21]:
tokenizer = create_tokenizer(all_data)
vocab_size = len(tokenizer.word_index) + 1

In [22]:
vocab_size

3304

In [23]:
len(post)

675

In [24]:
#POST ONLY AS TRAIN
data_X= encode_text(tokenizer, dataframe['snippet'], 16)
data_X_1= encode_text(tokenizer, dataframe['sentence'], 16)

In [25]:
trainX= encode_text(tokenizer, train_data['snippet'], 16)
trainX_1= encode_text(tokenizer, train_data['sentence'], 16)

In [26]:
head_X= encode_text(tokenizer, head_data['snippets'], 16)
head_X_1= encode_text(tokenizer, head_data['sentence'], 16)

post_X =encode_text(tokenizer, post_data['snippets'], 16)
post_X_1 =encode_text(tokenizer, post_data['sentence'], 16)


In [27]:
len(head_data['aspect'])

93

In [28]:
train_aspect = encode_text(tokenizer, train_data['aspect'],1)
train_aspect = np.tile(train_aspect,16)
head_aspect = encode_text(tokenizer, head_data['aspect'],1)
head_aspect = np.tile(head_aspect,16)
post_aspect = encode_text(tokenizer, post_data['aspect'],1)
post_aspect = np.tile(post_aspect,16)

# sentiment rescaling

In [29]:
def rescale(series,old_range,new_range):
    m = interp1d(old_range,new_range)
    return [float(m(x)) for x in series]

In [30]:
sentiment = rescale(train_data['sentiment'],[-1,1],[0,1])

In [31]:
h_sentiment= rescale (head_data['sentiment'],[-1,1],[0,1])
p_sentiment= rescale (post_data['sentiment'],[-1,1],[0,1])

# VALIDATION DATA

In [32]:
validation_X=[]
validation_aspect=[]
validation_Y=[]
for i in post_X:
    validation_X.append(i)
for i in head_X:
    validation_X.append(i)
    
for i in p_sentiment:
    validation_Y.append(i)
for i in h_sentiment:
    validation_Y.append(i) 
    
for i in post_aspect:
    validation_aspect.append(i)
for i in head_aspect:
    validation_aspect.append(i) 
    
validation_X=np.array(validation_X)
validation_aspect=np.array(validation_aspect)
validation_Y=np.array(validation_Y)



In [33]:
c = list(zip(validation_X,validation_aspect,validation_Y))
random.shuffle(c)
validation_X,validation_aspect,validation_Y= zip(*c)

In [34]:
    
validation_X=np.array(validation_X)
validation_aspect=np.array(validation_aspect)
validation_Y=np.array(validation_Y)


# WORD EMBEDDING

In [8]:

model = KeyedVectors.load_word2vec_format('D:\PythonCodes\Jupyter notebooks\Word Embeddings\GoogleNews-vectors-negative300.bin',limit=500000,binary=True)

pickle.dump(model,open("GoogleNews-vectors-negative300.dat","wb"))

In [37]:



def build_embedding_matrix(vocab_size, embed_dim, tokenizer ):
    
    embedding_matrix_file_name='D:/PythonCodes/Sentiment-Analysis/Data/embedding_matrix_sentiment_last.dat'
    
    if os.path.exists(embedding_matrix_file_name):
        
        print('loading embedding_matrix:', embedding_matrix_file_name)
        embedding_matrix = pickle.load(open(embedding_matrix_file_name, 'rb'))
        word2vec = pickle.load(open("word2vec.dat", 'rb'))

    else:
        

        embedding_matrix = np.zeros((vocab_size, embed_dim))
        word2vec={}
        for word, i in tokenizer.word_index.items():

            try:
                embedding_vector = model[word]
            except KeyError:
                embedding_vector = None
            if embedding_vector is not None:
                embedding_matrix[i]=embedding_vector
                word2vec[word]=i


        pickle.dump(embedding_matrix, open(embedding_matrix_file_name, 'wb'),protocol=2)
        pickle.dump(word2vec, open("word2vec.dat", 'wb'),protocol=2)


    return embedding_matrix,word2vec

In [38]:
embedding_matrix,w2v = build_embedding_matrix(vocab_size, 300,tokenizer)

loading embedding_matrix: D:/PythonCodes/Sentiment-Analysis/Data/embedding_matrix_sentiment_last.dat


In [39]:
print(len(embedding_matrix))

3304


In [40]:
len(embedding_matrix[w2v['company']])

300

In [41]:
#embedding_dict = gensim.models.KeyedVectors.load_word2vec_format(dictFileName, binary=False)
#embedding_dict.save_word2vec_format(dictFileName+".bin", binary=True)
#dictFileName="D:\wiki-news-300d-1M.vec\wiki-news-300d-1M.vec.bin"
#embedding_dict = gensim.models.KeyedVectors.load_word2vec_format(dictFileName, binary=True)

In [40]:
#embedding_dict

In [73]:
def get_fast_text_matrix(model):
    embedding_matrix = np.zeros((vocab_size,300))
    w2v={}
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = model[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
            w2v[word]=i

    return embedding_matrix,w2v

In [74]:
finance,w2v = get_fast_text_matrix(embedding_dict)


In [77]:
#pickle.dump(fast_text_model, open('finance_model.dat', 'wb'),protocol=2)

In [42]:
fast_text_model = pickle.load(open('fasttext_model.dat', 'rb'))
finance_model = pickle.load(open('finance_model.dat', 'rb'))

In [116]:
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r',errors='ignore')
    model = {}
    for line in f:
        splitLine = line.split()
        try: 
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except :
            pass
    print ("Done.",len(model)," words loaded!")
    return model
model = loadGloveModel(r"D:\glove.twitter.27B\glove.twitter.27B.200d.txt")

Loading Glove Model
Done. 1166543  words loaded!


In [117]:
pickle.dump(model, open('glove_model.dat', 'wb'))

In [118]:
model =pickle.load(open('glove_model.dat', 'rb'))

In [119]:
def get_glove_matrix(model):
    embedding_matrix = np.random.uniform(-0.001, 0.001, (vocab_size, 200))
    w2v={}
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = model[word]
        except KeyError:
            embedding_vector= None
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
            w2v[word]=i

    return embedding_matrix,w2v

glove,glove_w2v=get_glove_matrix(model)

In [120]:
vocab_size


3304

In [121]:
pickle.dump(glove, open('glove_embedding.dat', 'wb'))

In [43]:
glove = pickle.load(open('glove_embedding.dat', 'rb'))

# Make an embedding

In [50]:
import codecs

import multiprocessing
import os
import pprint
import re

import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['concatenate', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [51]:
all_data=[]
def fun(data,all_data):
    for row, i in data.iterrows():
        all_data.append(i['aspect'])
        all_data.append(i['sentence'])
    return all_data
    
all_data = fun(train_data,all_data)
all_data = fun(head_data,all_data)
all_data = fun(post_data,all_data)

In [52]:
#all_data = ' '.join(all_data)

In [53]:
def sentence_to_wordlist(raw):
    words = raw.split()
    return words

In [54]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = all_data
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [55]:
print(raw_sentences[101])
print(sentence_to_wordlist(raw_sentences[101]))

tesco set sell COMPANY giraffe businesses sky news
['tesco', 'set', 'sell', 'COMPANY', 'giraffe', 'businesses', 'sky', 'news']


In [56]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 13,138 tokens


In [57]:
num_features = 300
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 5

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling =1e-5

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

finance2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [58]:
finance2vec.build_vocab(sentences)

In [59]:
print("Word2Vec vocabulary length:", len(finance2vec.wv.vocab))

Word2Vec vocabulary length: 3337


In [60]:
finance2vec.train(sentences,total_examples=finance2vec.corpus_count,epochs=30)

(68331, 394140)

In [61]:
if not os.path.exists("trained"):
    os.makedirs("trained")
    
finance2vec.save(os.path.join("trained", "finance2vec.w2v"))

In [62]:
finance2vec = w2v.Word2Vec.load(os.path.join("trained", "finance2vec.w2v"))

In [63]:
finance_embedding = finance2vec.wv
embedding=finance_embedding.get_keras_embedding(train_embeddings=True)

In [64]:
len(finance2vec.wv.vocab)

3337

In [65]:
print(finance2vec.wv.index2word[0], finance2vec.wv.index2word[1], finance2vec.wv.index2word[2])


COMPANY Price Action


# LABEL ENCODING

In [44]:
from sklearn import preprocessing
from keras.utils import to_categorical 

def convert_lables (trainY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,27),le.classes_

In [45]:
trainY,lable_encoding = convert_lables(train_data['aspect'])
dataY,lable_encoding = convert_lables(dataframe['aspect'])

In [46]:
lable_encoding

array(['Appointment', 'Buyside', 'Central Banks', 'Company Communication',
       'Conditions', 'Coverage', 'Currency', 'Dividend Policy',
       'Financial', 'Fundamentals', 'IPO', 'Insider Activity', 'Legal',
       'M&A', 'Market', 'Options', 'Price Action', 'Regulatory',
       'Reputation', 'Risks', 'Rumors', 'Sales', 'Signal', 'Strategy',
       'Technical Analysis', 'Trade', 'Volatility'], dtype=object)

# LEXICONS


#### (1) OPINION LEXICON

In [47]:
with open(r"D:\lexicons\opinion-lexicon-English\negative-words.txt","r") as f: 
    lines= f.read()
    negative_words=lines.split("\n")
    for words in negative_words: 
        if words=='':
            negative_words.remove(words)
    negative_words=negative_words[:len(negative_words)-1]


with open(r"D:\lexicons\opinion-lexicon-English\positive-words.txt","r") as f: 
    lines= f.read()
    positive_words=lines.split("\n")
    for words in positive_words: 
        if words=='':
            positive_words.remove(words)
   

### (2) MRP


In [48]:
with open(r"D:\lexicons\MSOL-June15-09.txt\MSOL-June15-09.txt","r") as f: 
    lines= f.read()
    mrp_words=lines.split("\n")
    positive=[]
    negative=[]
    for words in mrp_words: 
        if words=='':
            mrp_words.remove(words)
        w=words.split(" ")
        try:
            if w[1]=='positive':
                positive.append(w[0])
            elif w[1]=='negative':
                negative.append(w[0])
            else: 
                print(w)
        except: 
            print(w)
    #negative_words=negative_words[:len(negative_words)-1]
    

['']


In [49]:
print(positive[100]),print(negative[100])

abjectly
a_sticky_moment


(None, None)

### (3) Loughran-McDonald

In [50]:
with open(r"D:\lexicons\LM_Negative.txt","r") as f: 
    lines= f.read()
    negative_words_1=lines.split("\n")
    n=[]
    for words in negative_words_1: 
        if words!='':
            n.append(words.lstrip('\x0c'))
    n=n[1:]
    n=n[:len(n)-1]

with open(r"D:\lexicons\LM_Positive.txt","r") as f: 
    lines= f.read()
    positive_words_1=lines.split("\n")
    p=[]
    for words in positive_words_1: 
        if words!='':
            p.append(words.lstrip('\x0c'))
    p=p[1:]
    p=p[:len(p)-1]


#### Combine All Lexicons

In [51]:
positive_w=[]
negative_w=[]
for i in positive: 
    positive_w.append(i.lower())
for i in p: 
    if i.lower() not in positive_w:
        positive_w.append(i.lower())    
for i in positive_words:
    if i.lower() not in positive_w:
        positive_w.append(i.lower())

for i in negative: 
    negative_w.append(i.lower())
for i in n: 
    if i.lower() not in negative_w:
        negative_w.append(i.lower())    
for i in negative_words:
    if i.lower() not in negative_w:
        negative_w.append(i.lower())


In [52]:
len(positive_w),len(negative_w)

(31250, 48649)

In [53]:
t=tokenizer
freq_word={}
for word, count in t.word_counts.items():
    freq_word[word]=count


# Sentence2vec

In [54]:
sentence2vec=[]
for i in train_data['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(embedding_matrix[w2v[word]])
            
        except:
            v=np.zeros((300),dtype=float)
            vec.append(v)
            
        
    sentence2vec.append(np.mean(vec,axis=0))


In [55]:
#sentence2vec

In [56]:
head_sentence2vec=[]
for i in head_data['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(embedding_matrix[w2v[word]])
            
        except:
            v=np.zeros((300),dtype=float)
            vec.append(v)
            
        
    head_sentence2vec.append(np.mean(vec,axis=0))
    
post_sentence2vec=[]
for i in post_data['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(embedding_matrix[w2v[word]])
            
        except:
            v=np.zeros((300),dtype=float)
            vec.append(v)
            
        
    post_sentence2vec.append(np.mean(vec,axis=0))


In [57]:
len(post_sentence2vec),len(head_sentence2vec)

(99, 93)

# GLOVE

In [58]:
sentence2vec_glove=[]
for i in train_data['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(glove[glove_w2v[word]])
            
        except:
            v=np.zeros((200),dtype=float)
            vec.append(v)
            
        
    sentence2vec_glove.append(np.mean(vec,axis=0))


In [59]:
sentence2vec_glove_post=[]
for i in post['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(glove[glove_w2v[word]])
            
        except:
            v=np.zeros((200),dtype=float)
            vec.append(v)
            
        
    sentence2vec_glove_post.append(np.mean(vec,axis=0))

In [60]:
head_sentence2vec_glove=[]
for i in head_data['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(glove[glove_w2v[word]])
            
        except:
            v=np.zeros((200),dtype=float)
            vec.append(v)
            
        
    head_sentence2vec_glove.append(np.mean(vec,axis=0))
    
post_sentence2vec_glove=[]
for i in post_data['sentence']:
    vec=[]
    for word in i.lower().split(" "):
        try:
            vec.append(glove[glove_w2v[word]])
            
        except:
            v=np.zeros((200),dtype=float)
            vec.append(v)
            
        
    post_sentence2vec_glove.append(np.mean(vec,axis=0))


#  Calculating PMI SCORE

In [61]:
train_data['sentiment']=train_data['sentiment'].astype('float')

In [62]:
from collections import Counter
import math

In [63]:
def cal_pmi_score(data=train_data):
    freq_w_pos={}
    freq_w_neg={}
    positive_sentences=[]
    negative_sentences=[]
    N=vocab_size
    #freq_w= Counter(data['sentence'])
    
    for index,row in data.iterrows():

        if row['sentiment']>0:
           
            #row['Sentiment Class']='POSITIVE'
            positive_sentences.append(row['sentence'])
            for i in row['sentence'].split(" "):
                if i.lower() not in freq_w_pos.keys():
                    freq_w_pos[i.lower()]=1
                else :
                    freq_w_pos[i.lower()]+=1



        elif row['sentiment']<0:
           
            #row['Sentiment Class']='NEGATIVE'
            negative_sentences.append(row['sentence'])
            for i in row['sentence'].split(" "):
                if i.lower() not in freq_w_neg.keys():
                    freq_w_neg[i.lower()]=1
                else :
                    freq_w_neg[i.lower()]+=1

        else : 
            pass
            
        
    t1=create_tokenizer(positive_sentences)

    t2=create_tokenizer(negative_sentences)

    freq_pos={}
    freq_neg={}
    for word, count in t1.word_counts.items():
        freq_pos[word]=count
    for word, count in t2.word_counts.items():
        freq_neg[word]=count
    tp = len(freq_pos)
    tn=  len(freq_neg)

    score_w={}
    
    for i in t.word_counts.keys():
       
        try: 
                a=freq_w_pos[i]
        except :
                a=0
                
        try:
                c=freq_w_neg[i]
        except:
                c=0
    
            
         
        cal_1=(a * N)/(freq_word[i] * tp)
        try : pmi_w_pos = math.log(cal_1,2)
        except: pmi_w_pos=0
        cal_2=(c * N)/(freq_word[i] * tn)
        try : pmi_w_neg = math.log(cal_2,2)
        except : pmi_w_neg=0
        score_w[i]= pmi_w_pos-pmi_w_neg

    return score_w
                
                
            
score = cal_pmi_score(train_data)        
     

In [64]:
semantic_sorted = sorted(score.items(), 
                         reverse=True)
top_pos = semantic_sorted[30:40]
top_neg = semantic_sorted[-30:-20]
print(top_neg)
print(top_pos)

[('accumulating', 0.00240358184332116), ('accumulate', 0.5873660825644773), ('accrue', 0.5873660825644773), ('accident', 0), ('access', 0), ('accelerate', 0.5873660825644773), ('academics', -1.1287100262546164), ('above', 0), ('abc', 0.5873660825644773), ('abandons', -1.1287100262546164)]
[('yib', 0.5873660825644773), ('yetwhen', -1.1287100262546164), ('yet', -1.126306444411295), ('yesterdaylooking', 0.5873660825644773), ('yesterday', 1.7805841511972234), ('yes', 0.5873660825644773), ('years', 0.4586560563098611), ('year', -0.01082922699135902), ('yeah', -1.1287100262546164), ('ybxeze', -1.1287100262546164)]


In [65]:
SCORE_HEAD =  cal_pmi_score(head_data)
SCORE_POST =  cal_pmi_score(post_data)


In [66]:
len(score)

3303

In [67]:
pos_count=[]
neg_count=[]
net_count=[]
a_score=[]
pmi_score=[]
for i in train_data['sentence']:
    p_count=0
    n_count=0
    p=[]
    for j in i.lower().split(" "):
        if j in positive_w:
            p_count+=1
        if j in negative_w:
            n_count+=1
        try : p.append(score[j])
        except : p.append(0)
    pmi_score.append((np.mean(p)))
    a=1-np.sqrt(1-((p_count-n_count)/(p_count+n_count)))
    pos_count.append(p_count)
    neg_count.append(n_count)
    net_count.append(p_count-n_count)
    a_score.append(a)
    


In [68]:
train_data['positive']=pos_count
train_data['negative']=neg_count
train_data['net']=net_count
train_data['a_score']=a_score
train_data['pmi']= pmi_score

In [69]:
train_data.head()

,aspect,sentence,sentiment,snippet,positive,negative,net,a_score,pmi
0,Financial,linkedin revenue nearly f COMPANY,0.423,latest bid said unlikely to win sabmiller's ap...,3,0,3,1.000000,-0.456496
1,Risks,aviva COMPANY suspend property funds investors...,-0.807,"shakes up board with two new business chiefs, ...",3,2,1,0.105573,-0.399879
2,Price Action,britain ftse bounces back mondi COMPANY lead,0.558,"set to appoint roy gardner, ex-centrica, as ch...",3,1,2,0.292893,0.676420
3,Risks,new COMPANY duo get former boss support diffus...,0.143,"set to appoint roy gardner, ex-centrica, as ch...",6,4,2,0.105573,0.129499
4,Market,amazon attack uk grocery market COMPANY deal,0.333,sells benchmark indices unit,2,4,-2,-0.154701,0.246706


In [70]:
pos_count=[]
neg_count=[]
net_count=[]
a_score=[]
h_pmi_score=[]
for i in head_data['sentence']:
    p_count=0
    n_count=0
    p=[]
    for j in i.lower().split(" "):
        if j in positive_w:
            p_count+=1
        if j in negative_w:
            n_count+=1
        try : p.append(score[j])
        except : p.append(0)
    h_pmi_score.append((np.mean(p)))
    a=1-np.sqrt(1-((p_count-n_count)/(p_count+n_count)))
    pos_count.append(p_count)
    neg_count.append(n_count)
    net_count.append(p_count-n_count)
    a_score.append(a)

head_data['positive']=pos_count
head_data['negative']=neg_count
head_data['net']=net_count
head_data['a_score']=a_score
head_data['pmi']=h_pmi_score
head_data.head()

,aspect,sentence,sentiment,snippets,positive,negative,net,a_score,pmi
0,Financial,COMPANY delivers small profit increase 2014,0.247,delivers small profit increase for,3,1,2,0.292893,0.366023
1,Financial,COMPANY reports million loss first quarter,-0.741,reports $583 million loss in first quarter,2,2,0,0.000000,-0.212220
2,Sales,osborne extends COMPANY sell plan,-0.311,sell-off plan,1,2,-1,-0.154701,-0.338741
3,Price Action,companies COMPANY shares hit 2016 revenue fore...,-0.375,hit as 2016 revenue forecast to fall,3,2,1,0.105573,-0.244071
4,Strategy,COMPANY ends 27 year sponsorship tate falling ...,-0.293,as falling oil price takes toll,4,5,-1,-0.054093,-0.290480


In [71]:
pos_count=[]
neg_count=[]
net_count=[]
a_score=[]
p_pmi_score=[]
for i in post_data['sentence']:
    p_count=0
    n_count=0
    for j in i.lower().split(" "):
        if j in positive_w:
            p_count+=1
        if j in negative_w:
            n_count+=1
        try : p.append(score[j])
        except : p.append(0)
    p_pmi_score.append((np.mean(p)))
    a=1-np.sqrt(1-((p_count-n_count)/(p_count+n_count)))
    pos_count.append(p_count)
    neg_count.append(n_count)
    net_count.append(p_count-n_count)
    a_score.append(a)

post_data['positive']=pos_count
post_data['negative']=neg_count
post_data['net']=net_count
post_data['a_score']=a_score
post_data['pmi']=p_pmi_score

post_data.head()

,aspect,sentence,sentiment,snippets,positive,negative,net,a_score,pmi
0,Price Action,COMPANY currently way undervalued rise back soon,0.154,is currently way undervalued and will rise bac...,4,3,1,0.074180,-0.082579
1,Financial,COMPANY q delivered organic revenue de growth...,0.408,reported net revenue up 2.0%,9,3,6,0.292893,-0.101138
2,Price Action,one stock oversold rsi watch possible reverse...,0.000,stocks oversold rsi 20,4,2,2,0.183503,0.025945
3,Price Action,COMPANY added dip yesterday rick holding night...,0.000,but i’m running away from this.,4,4,0,0.000000,0.125071
4,Financial,COMPANY revenue falls lowers full year revenu...,-0.950,revenue falls,5,2,3,0.244071,0.040820


# EXTRACTING N GRAMS 

In [72]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy import sparse

# (1) COUNT VECTORIZER

In [73]:
count_vect = CountVectorizer(analyzer='word')
count_vect.fit(dataframe['sentence'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_data['sentence'])
xtrain_1_count =  count_vect.transform(dataframe['sentence'])
head_count =  count_vect.transform(head_data['sentence'])
post_count =  count_vect.transform(post_data['sentence'])

In [74]:
xtrain_count

<1173x3023 sparse matrix of type '<class 'numpy.int64'>'
	with 9213 stored elements in Compressed Sparse Row format>

# (2) TFID

In [75]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=None)
tfidf_vect.fit(dataframe['sentence'])
xtrain_tfidf =  tfidf_vect.transform(train_data['sentence'])
xtrain_1_tfidf =  tfidf_vect.transform(dataframe['sentence'])
head_tfidf =  tfidf_vect.transform(head_data['sentence'])
post_tfidf =  tfidf_vect.transform(post_data['sentence'])


# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,4), max_features=None)
tfidf_vect_ngram.fit(train_data['sentence'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_data['sentence'])
head_tfidf_ngram =  tfidf_vect_ngram.transform(head_data['sentence'])
post_tfidf_ngram =  tfidf_vect_ngram.transform(post_data['sentence'])
xtrain_1_tfidf_ngram =  tfidf_vect_ngram.transform(dataframe['sentence'])

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(1,3), max_features=None)
tfidf_vect_ngram_chars.fit(train_data['sentence'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_data['sentence']) 
head_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(head_data['sentence']) 
post_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(post_data['sentence']) 
xtrain_1_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(dataframe['sentence']) 


# Features for Aspect

In [76]:
import textblob

In [77]:
train_data['char_count'] = train_data['sentence'].apply(len)
train_data['word_count'] = train_data['sentence'].apply(lambda x: len(x.split()))
train_data['word_density'] = train_data['char_count'] / (train_data['word_count']+1)

In [78]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

train_data['noun_count'] = train_data['sentence'].apply(lambda x: check_pos_tag(x, 'noun'))
train_data['verb_count'] = train_data['sentence'].apply(lambda x: check_pos_tag(x, 'verb'))
train_data['adj_count'] = train_data['sentence'].apply(lambda x: check_pos_tag(x, 'adj'))
train_data['adv_count'] = train_data['sentence'].apply(lambda x: check_pos_tag(x, 'adv'))
train_data['pron_count'] = train_data['sentence'].apply(lambda x: check_pos_tag(x, 'pron'))

In [79]:

post['noun_count'] = post['sentence'].apply(lambda x: check_pos_tag(x, 'noun'))
post['verb_count'] = post['sentence'].apply(lambda x: check_pos_tag(x, 'verb'))
post['adj_count'] = post['sentence'].apply(lambda x: check_pos_tag(x, 'adj'))
post['adv_count'] = post['sentence'].apply(lambda x: check_pos_tag(x, 'adv'))
post['pron_count'] = post['sentence'].apply(lambda x: check_pos_tag(x, 'pron'))

In [80]:
head_data['noun_count'] = head_data['sentence'].apply(lambda x: check_pos_tag(x, 'noun'))
head_data['verb_count'] = head_data['sentence'].apply(lambda x: check_pos_tag(x, 'verb'))
head_data['adj_count'] = head_data['sentence'].apply(lambda x: check_pos_tag(x, 'adj'))
head_data['adv_count'] = head_data['sentence'].apply(lambda x: check_pos_tag(x, 'adv'))
head_data['pron_count'] = head_data['sentence'].apply(lambda x: check_pos_tag(x, 'pron'))

In [81]:
post_data['noun_count'] = post_data['sentence'].apply(lambda x: check_pos_tag(x, 'noun'))
post_data['verb_count'] = post_data['sentence'].apply(lambda x: check_pos_tag(x, 'verb'))
post_data['adj_count'] = post_data['sentence'].apply(lambda x: check_pos_tag(x, 'adj'))
post_data['adv_count'] = post_data['sentence'].apply(lambda x: check_pos_tag(x, 'adv'))
post_data['pron_count'] = post_data['sentence'].apply(lambda x: check_pos_tag(x, 'pron'))

In [82]:
head_data.head()

,aspect,sentence,sentiment,snippets,positive,negative,net,a_score,pmi,noun_count,verb_count,adj_count,adv_count,pron_count
0,Financial,COMPANY delivers small profit increase 2014,0.247,delivers small profit increase for,3,1,2,0.292893,0.366023,3,1,1,0,0
1,Financial,COMPANY reports million loss first quarter,-0.741,reports $583 million loss in first quarter,2,2,0,0.000000,-0.212220,3,1,1,0,0
2,Sales,osborne extends COMPANY sell plan,-0.311,sell-off plan,1,2,-1,-0.154701,-0.338741,3,1,1,0,0
3,Price Action,companies COMPANY shares hit 2016 revenue fore...,-0.375,hit as 2016 revenue forecast to fall,3,2,1,0.105573,-0.244071,5,2,0,0,0
4,Strategy,COMPANY ends 27 year sponsorship tate falling ...,-0.293,as falling oil price takes toll,4,5,-1,-0.054093,-0.290480,7,3,0,0,0


# All Features

### Sentiment Models

In [83]:
X = sparse.hstack([xtrain_tfidf, xtrain_tfidf_ngram,xtrain_tfidf_ngram_chars,sparse.csr_matrix(train_data['a_score']).T,sparse.csr_matrix(train_data['net']).T,sparse.csr_matrix(train_data['negative']).T,sparse.csr_matrix(train_data['positive']).T,sparse.csr_matrix(train_data['pmi']).T],'csr')
x_test_1 = sparse.hstack([head_tfidf, head_tfidf_ngram,head_tfidf_ngram_chars,sparse.csr_matrix(head_data['a_score']).T,sparse.csr_matrix(head_data['net']).T,sparse.csr_matrix(head_data['negative']).T,sparse.csr_matrix(head_data['positive']).T,sparse.csr_matrix(head_data['pmi']).T],'csr')
x_test_2 = sparse.hstack([post_tfidf, post_tfidf_ngram,post_tfidf_ngram_chars,sparse.csr_matrix(post_data['a_score']).T,sparse.csr_matrix(post_data['net']).T,sparse.csr_matrix(post_data['negative']).T,sparse.csr_matrix(post_data['positive']).T,sparse.csr_matrix(post_data['pmi']).T],'csr')


### Aspect Models

In [84]:
'''
X_aspect = sparse.hstack([xtrain_tfidf, xtrain_tfidf_ngram,xtrain_tfidf_ngram_chars,
                   sparse.csr_matrix(train_data['noun_count']).T,sparse.csr_matrix(train_data['verb_count']).T,
                   sparse.csr_matrix(train_data['adj_count']).T,sparse.csr_matrix(train_data['adv_count']).T,
                   sparse.csr_matrix(train_data['pron_count']).T,X_topics,
                   sparse.csr_matrix(train_data['a_score']).T,sparse.csr_matrix(train_data['net']).T,
                   sparse.csr_matrix(train_data['negative']).T,sparse.csr_matrix(train_data['positive']).T,
                   sparse.csr_matrix(train_data['pmi']).T],'csr')

'''
X_aspect = sparse.hstack([xtrain_count,xtrain_tfidf, xtrain_tfidf_ngram,xtrain_tfidf_ngram_chars],'csr')
X_aspect_1 = sparse.hstack([xtrain_1_count,xtrain_1_tfidf, xtrain_1_tfidf_ngram,xtrain_1_tfidf_ngram_chars],'csr')

In [85]:
'''
x_test_1_aspect = sparse.hstack([head_tfidf, head_tfidf_ngram,head_tfidf_ngram_chars,
                  sparse.csr_matrix(head_data['noun_count']).T,sparse.csr_matrix(head_data['verb_count']).T,
                   sparse.csr_matrix(head_data['adj_count']).T,sparse.csr_matrix(head_data['adv_count']).T,
                   sparse.csr_matrix(head_data['pron_count']).T,Head_topics,sparse.csr_matrix(head_data['a_score']).T,
                   sparse.csr_matrix(head_data['net']).T,sparse.csr_matrix(head_data['negative']).T,
                sparse.csr_matrix(head_data['positive']).T,sparse.csr_matrix(head_data['pmi']).T],'csr')
'''
x_test_1_aspect = sparse.hstack([head_count,head_tfidf, head_tfidf_ngram,head_tfidf_ngram_chars],'csr')

In [86]:
'''
x_test_2_aspect = sparse.hstack([post_tfidf, post_tfidf_ngram,post_tfidf_ngram_chars,
                   sparse.csr_matrix(post_data['noun_count']).T,sparse.csr_matrix(post_data['verb_count']).T,
                   sparse.csr_matrix(post_data['adj_count']).T,sparse.csr_matrix(post_data['adv_count']).T,
                   sparse.csr_matrix(post_data['pron_count']).T,Post_topics,
                 sparse.csr_matrix(post_data['a_score']).T,sparse.csr_matrix(post_data['net']).T,sparse.csr_matrix(post_data['negative']).T,
                sparse.csr_matrix(post_data['positive']).T,sparse.csr_matrix(post_data['pmi']).T],'csr')
'''

x_test_2_aspect = sparse.hstack([post_count,post_tfidf, post_tfidf_ngram,post_tfidf_ngram_chars],'csr')

In [87]:
X_s2v=sparse.csc_matrix(sentence2vec)
X_h2v=sparse.csc_matrix(head_sentence2vec)
X_p2v=sparse.csc_matrix(post_sentence2vec)

X_s2v_g=sparse.csc_matrix(sentence2vec_glove)
X_h2v_g=sparse.csc_matrix(head_sentence2vec_glove)
X_p2v_g=sparse.csc_matrix(post_sentence2vec_glove)


# Interactive Model 2

In [88]:
activator = Activation('relu', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [89]:
def one_step_attention(h, avg):
    print("H",h.shape)
    print("avg before: ",avg.shape)
    avg = RepeatVector(max_length)(avg)
    print("avg",avg.shape)
    
    concat = concatenate([h, avg])
    print("concat",concat.shape)
    
    e = Dense(1,input_dim=concat.shape, activation = "relu")(concat)
    print("e",e.shape)

    alphas = activator(e)
    print("alphas",alphas.shape)

    context = dotor([alphas, h])
    print("context",context.shape)
    
    return K.sum(context, axis=1)

In [258]:

from math import*
 
def square_rooted(x):
 
    return round(sqrt(sum([a*a for a in x])),3)
 
def cosine_similarity(x,y):
 
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return numerator/float(denominator)

In [124]:
trainX.shape

(1173, 16)

In [266]:

def Model_IAN(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag, em_dim,input_shape):
    
    input_context = Input(shape=(max_length,),name='Context')
    input_features = Input(shape=(input_shape,),name='Features')
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)

    context=embedding(input_context)
 
    H_c,_,_,_,_= Bidirectional(LSTM( lstm_out, recurrent_dropout=dropout,return_state=True,return_sequences=True,name="LSTM_C"))(context)

    c_avg = GlobalAveragePooling1D(name='POOL_C')(H_c)  

    c_r = Lambda(lambda x: one_step_attention(x[0],x[1]))([H_c,input_features])
   
    out= Dense(327, activation='relu')(c_r)

    out= Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(out)

    IAN_model= Model(inputs=[input_context,input_features],outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    IAN_model.compile(loss='mse', optimizer=optimizer, metrics=['cosine'])
    
    #print( IAN_model.summary())
    
    return  IAN_model
   

In [267]:
max_length = 16

In [268]:
IAN_model=Model_IAN(learning_rate=0.01,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='glove',
                     em_trainable_flag=False,
                     em_dim=300,
                   input_shape=X.shape[1])

H (?, ?, 600)
avg before:  (?, 27380)
avg (?, 16, 27380)
concat (?, 16, 27980)
e (?, 16, 1)
alphas (?, 16, 1)
context (?, 1, 600)
H (?, 16, 600)
avg before:  (?, 27380)
avg (?, 16, 27380)
concat (?, 16, 27980)
e (?, 16, 1)
alphas (?, 16, 1)
context (?, 1, 600)


In [271]:
EarlyStop= EarlyStopping(monitor='val_loss',patience=5,verbose=1)


IAN_model.fit(x=[trainX,X],y=sentiment, epochs=100,batch_size=16,
              callbacks=[EarlyStop],validation_data=([head_X,x_test_1],h_sentiment))

Train on 1173 samples, validate on 93 samples
Epoch 1/100
1173/1173 [==============================] - 13s 11ms/step - loss: 0.3549 - cosine_proximity: -8.5251e-04 - val_loss: 0.3113 - val_cosine_proximity: 0.0000e+00
Epoch 2/100
1173/1173 [==============================] - 13s 11ms/step - loss: 0.3544 - cosine_proximity: -8.5252e-04 - val_loss: 0.3110 - val_cosine_proximity: -5.8674e-21
Epoch 3/100
1173/1173 [==============================] - 13s 11ms/step - loss: 0.3546 - cosine_proximity: -2.0747e-09 - val_loss: 0.3109 - val_cosine_proximity: -1.0802e-14
Epoch 4/100
1173/1173 [==============================] - 14s 12ms/step - loss: 0.3545 - cosine_proximity: -6.8815e-06 - val_loss: 0.3108 - val_cosine_proximity: -1.1199e-07
Epoch 5/100
1173/1173 [==============================] - 13s 11ms/step - loss: 0.3088 - cosine_proximity: -0.6064 - val_loss: 0.3804 - val_cosine_proximity: -1.0000
Epoch 6/100
1173/1173 [==============================] - 13s 11ms/step - loss: 0.2937 - cosine_pro

KeyboardInterrupt: 

In [272]:

pred_sentiment =IAN_model.predict([head_X,x_test_1])

outputs= sk_mse(h_sentiment,pred_sentiment)

output1= r2_score(h_sentiment,pred_sentiment)

print("Cosine : ",cosine_similarity(h_sentiment,list(pred_sentiment)))

print("Headline MSE: ", np.mean(outputs))
print("Headline R2:", np.mean(output1))

pred_sentiment =IAN_model.predict([post_X,x_test_2])

#pred_sentiment = rescale(pred_sentiment,[0,1],[-1,1])

outputs= sk_mse(p_sentiment,pred_sentiment)

output1= r2_score(p_sentiment,pred_sentiment)
print("Cosine : ",cosine_similarity(p_sentiment,pred_sentiment))

print("Post MSE:", np.mean(outputs))
print("Post R2:", np.mean(output1))



Cosine :  [0.8945746]
Headline MSE:  0.06613861480577815
Headline R2: -0.06623033857550586
Cosine :  [0.95096034]
Post MSE: 0.030428054894036563
Post R2: -0.06134750833404512


#  Trying Different Models

In [362]:
def train_model(model, feature_vector_train, label, feature_vector_valid,feature_vector_valid1, is_neural_net=False):
    # fit the training dataset on the classifier
    if is_neural_net: 
        EarlyStop= EarlyStopping(monitor='val_loss',patience=5,verbose=1)
        model.fit(feature_vector_train, label,batch_size=16, epochs=50,callbacks=[EarlyStop],validation_data=(feature_vector_valid,h_sentiment))
    else:
        model.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = model.predict(feature_vector_valid)
    predictions1 = model.predict(feature_vector_valid1)
    predictions2 = model.predict(feature_vector_train)
   

    print("Cosine : ",cosine_similarity(h_sentiment,list(predictions)))

    print("Headline MSE: ", np.mean(sk_mse(h_sentiment,predictions)))
    print("Headline R2:", np.mean(r2_score(h_sentiment,predictions)))
    print()
    print("Cosine : ",cosine_similarity(p_sentiment,predictions1))

    print("Headline MSE: ", np.mean(sk_mse(p_sentiment,predictions1)))
    print("Headline R2:", np.mean(r2_score(p_sentiment,predictions1)))
    
    return model,predictions,predictions1,predictions2

# Linear Regression

In [363]:
from sklearn import model_selection, preprocessing, linear_model, metrics, svm


In [365]:
from sklearn.linear_model import LinearRegression

# Linear Classifier on Count Vectors
print ("LR, Count Vectors: ")
m,m_A,m_B,_=train_model( LinearRegression() , xtrain_count, np.array(sentiment), head_count,post_count)

print()
# Linear Classifier on Word Level TF IDF Vectors
print ("LR, WordLevel TF-IDF: ")
m1,m_1_A,m_1_B,m_1_C=train_model( LinearRegression(), xtrain_tfidf, np.array(sentiment), head_tfidf,post_tfidf)
print()
# Linear Classifier on Ngram Level TF IDF Vectors
print ("LR, N-Gram Vectors: ")
m3,m_3_A,m_3_B,m_3_C=train_model( LinearRegression(), xtrain_tfidf_ngram, np.array(sentiment), head_tfidf_ngram, post_tfidf_ngram)
print()
# Linear Classifier on Character Level TF IDF Vectors
print ("LR, CharLevel Vectors: ")
m4,m_4_A,m_4_B,_=train_model(LinearRegression(), xtrain_tfidf_ngram_chars,np.array(sentiment) ,head_tfidf_ngram_chars, post_tfidf_ngram_chars)


LR, Count Vectors: 
Cosine :  0.8847039394020311
Headline MSE:  0.08638859655412545
Headline R2: -0.39268327320518437

Cosine :  0.923343053803146
Headline MSE:  0.07436840787624092
Headline R2: -1.5940115026441437

LR, WordLevel TF-IDF: 
Cosine :  0.8977764918508906
Headline MSE:  0.0696291217340215
Headline R2: -0.12250131423518829

Cosine :  0.9331148702849251
Headline MSE:  0.0476506109473248
Headline R2: -0.6620798593816737

LR, N-Gram Vectors: 
Cosine :  0.8985828709608396
Headline MSE:  0.06710067706940412
Headline R2: -0.08173988585116221

Cosine :  0.9523212162259722
Headline MSE:  0.032520733338035165
Headline R2: -0.13434129844047638

LR, CharLevel Vectors: 
Cosine :  0.8959732968556327
Headline MSE:  0.07951853164029304
Headline R2: -0.28192994611144395

Cosine :  0.9273612455048713
Headline MSE:  0.054432656293780066
Headline R2: -0.8986413798250827


In [366]:
from sklearn.svm import SVR
import numpy as np
np.random.seed(0)
clf = SVR(C=1.0, epsilon=0.2)
a=clf.fit(X, sentiment) 

In [367]:
pred2=clf.predict(X)
pred=clf.predict( x_test_1)
pred1=clf.predict( x_test_2)
print(sk_mse(sentiment,pred2))
print(sk_mse(h_sentiment,pred))
print(sk_mse(p_sentiment,pred1))

0.036675339644856725
0.06030041562885621
0.028242021213239592


In [368]:
print("C = ", r2_score(np.array(p_sentiment).reshape(-1,1),np.array(pred1).reshape(-1,1)))
print("C = ", r2_score(np.array(h_sentiment).reshape(-1,1),np.array(pred).reshape(-1,1)))

C =  0.014902564446745203
C =  0.02788812918134953


In [369]:
print("C = ", cosine_similarity(np.array(p_sentiment).reshape(-1,1),np.array(pred1).reshape(-1,1)))
print("C = ", cosine_similarity(np.array(h_sentiment).reshape(-1,1),np.array(pred).reshape(-1,1)))


C =  [0.95217958]
C =  [0.89999031]


# GHOSAL MODEL

### (1) FEATURE MODEL - Only with tfid_n_gram

In [370]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = Dense(500, activation="relu")(input_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    
    hidden_layer = Dense(50, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    hidden_layer = Dense(20, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    output_layer = Dense(1, activation="sigmoid")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)


    classifier = Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=Adam(lr=0.001), loss='mse')
    return classifier

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])


In [372]:
_,m1,m2,m3=train_model(classifier, xtrain_tfidf_ngram, sentiment, head_tfidf_ngram, post_tfidf_ngram, is_neural_net=True)

Train on 1173 samples, validate on 93 samples
Epoch 1/50
1173/1173 [==============================] - 16s 14ms/step - loss: 0.0399 - val_loss: 0.0676
Epoch 2/50
1173/1173 [==============================] - 16s 14ms/step - loss: 0.0177 - val_loss: 0.0658
Epoch 3/50
1173/1173 [==============================] - 16s 14ms/step - loss: 0.0139 - val_loss: 0.0664
Epoch 4/50
1173/1173 [==============================] - 16s 14ms/step - loss: 0.0073 - val_loss: 0.0682
Epoch 5/50
1173/1173 [==============================] - 16s 13ms/step - loss: 0.0048 - val_loss: 0.0637
Epoch 6/50
1173/1173 [==============================] - 17s 14ms/step - loss: 0.0038 - val_loss: 0.0633
Epoch 7/50
1173/1173 [==============================] - 16s 14ms/step - loss: 0.0031 - val_loss: 0.0641
Epoch 8/50
1173/1173 [==============================] - 16s 13ms/step - loss: 0.0031 - val_loss: 0.0631
Epoch 9/50
1173/1173 [==============================] - 16s 13ms/step - loss: 0.0029 - val_loss: 0.0662
Epoch 10/50
1173/1

### (1) FEATURE MODEL - Pipelining

In [373]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = Dense(500, activation="relu")(input_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    
    hidden_layer = Dense(50, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    hidden_layer = Dense(20, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    output_layer = Dense(1, activation="sigmoid")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)


    classifier = Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=Adam(lr=0.001), loss='mse')
    
    return classifier


classifier = create_model_architecture(X.shape[1])
model1,m6_A,m6_B,m6_C=train_model(classifier, X, sentiment, x_test_1, x_test_2, is_neural_net=True)

Train on 1173 samples, validate on 93 samples
Epoch 1/50
1173/1173 [==============================] - 28s 24ms/step - loss: 0.0289 - val_loss: 0.0526
Epoch 2/50
1173/1173 [==============================] - 23s 20ms/step - loss: 0.0132 - val_loss: 0.0562
Epoch 3/50
1173/1173 [==============================] - 21s 18ms/step - loss: 0.0075 - val_loss: 0.0529
Epoch 4/50
1173/1173 [==============================] - 20s 17ms/step - loss: 0.0053 - val_loss: 0.0551
Epoch 5/50
1173/1173 [==============================] - 19s 16ms/step - loss: 0.0047 - val_loss: 0.0587
Epoch 6/50
1173/1173 [==============================] - 18s 16ms/step - loss: 0.0037 - val_loss: 0.0558
Epoch 00006: early stopping
Cosine :  [0.91153574]
Headline MSE:  0.07998227890170187
Headline R2: 0.10009056558596485

Cosine :  [0.9582718]
Headline MSE:  0.039825587368043376
Headline R2: 0.016856415050312323


### (2) Vector Averaging

In [389]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = Dense(500, activation="relu")(input_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    
    hidden_layer = Dense(50, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    hidden_layer = Dense(20, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    output_layer = Dense(1, activation="sigmoid")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)


    classifier = Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=Adam(lr=0.001), loss='mse')
    
    return classifier


classifier = create_model_architecture(X_s2v.shape[1])
model2,m7_a,m7_b,m7_c=train_model(classifier, X_s2v, sentiment,X_h2v, X_p2v, is_neural_net=True)

Train on 1173 samples, validate on 93 samples
Epoch 1/50
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0384 - val_loss: 0.0580
Epoch 2/50
1173/1173 [==============================] - 1s 823us/step - loss: 0.0309 - val_loss: 0.0553
Epoch 3/50
1173/1173 [==============================] - 1s 809us/step - loss: 0.0258 - val_loss: 0.0469
Epoch 4/50
1173/1173 [==============================] - 1s 784us/step - loss: 0.0230 - val_loss: 0.0442
Epoch 5/50
1173/1173 [==============================] - 1s 783us/step - loss: 0.0207 - val_loss: 0.0428
Epoch 6/50
1173/1173 [==============================] - 1s 752us/step - loss: 0.0176 - val_loss: 0.0451
Epoch 7/50
1173/1173 [==============================] - 1s 748us/step - loss: 0.0158 - val_loss: 0.0421
Epoch 8/50
1173/1173 [==============================] - 1s 758us/step - loss: 0.0133 - val_loss: 0.0476
Epoch 9/50
1173/1173 [==============================] - 1s 820us/step - loss: 0.0117 - val_loss: 0.0408
Epoch 10/50
1173/117

### (3) CNN 

In [399]:
def define_model2(dropout,learning_rate,em,em_dim,lstm_out, n_hidden_layer,em_trainable_flag,n_filters=100):
   
    input_context= Input(shape=(max_length,),name='Context')

    embedding1=Embedding(vocab_size, em_dim, weights = [eval(em)],input_length=max_length,trainable = False)
    
    context1= embedding1(input_context)
    
    concat1=Dropout(0.3)(context1)
    
    c3=Conv1D(n_filters,kernel_size=3,activation='relu')(concat1)
    drop3 = Dropout(0.2)(c3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat = Flatten()(pool3)
    
    a=Dense(327,activation='relu')(flat)
        
    out=Dense(1,activation='sigmoid')(a)
    
    model= Model(inputs=input_context ,outputs=out)
    

    optimizer = Adam(lr=learning_rate)

    model.compile(loss='mse', optimizer=optimizer, metrics=['cosine'])
    
    
    #print(model.summary())
    
    return model

In [400]:
model=define_model2(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='glove',
                     em_trainable_flag=False,
                     em_dim=200)




In [401]:
EarlyStop= EarlyStopping(monitor='val_loss',patience=5,verbose=1)

model.fit(x=trainX,y=sentiment, epochs=100,batch_size=16,callbacks=[EarlyStop],validation_data=(validation_X,validation_Y))

Train on 1173 samples, validate on 192 samples
Epoch 1/100
1173/1173 [==============================] - 5s 4ms/step - loss: 0.0462 - cosine_proximity: -1.0000 - val_loss: 0.0513 - val_cosine_proximity: -1.0000
Epoch 2/100
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0383 - cosine_proximity: -1.0000 - val_loss: 0.0467 - val_cosine_proximity: -1.0000
Epoch 3/100
1173/1173 [==============================] - 1s 956us/step - loss: 0.0346 - cosine_proximity: -1.0000 - val_loss: 0.0469 - val_cosine_proximity: -1.0000
Epoch 4/100
1173/1173 [==============================] - 1s 993us/step - loss: 0.0313 - cosine_proximity: -1.0000 - val_loss: 0.0472 - val_cosine_proximity: -1.0000
Epoch 5/100
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0292 - cosine_proximity: -1.0000 - val_loss: 0.0523 - val_cosine_proximity: -1.0000A: 0s - loss: 0.0292 - cosine_proximity: -
Epoch 6/100
1173/1173 [==============================] - 1s 1ms/step - loss: 0.0276 - cosine

In [402]:

pred_sentiment =model.predict(head_X)

outputs= sk_mse(h_sentiment,pred_sentiment)

output1= r2_score(h_sentiment,pred_sentiment)
print("C = ", cosine_similarity(np.array(h_sentiment).reshape(-1,1),np.array(pred_sentiment).reshape(-1,1)))

print("Headline MSE: ", np.mean(outputs))
print("Headline R2:", np.mean(output1))

pred_sentiment1 =model.predict(post_X)

print("C = ", cosine_similarity(np.array(p_sentiment).reshape(-1,1),np.array(pred_sentiment1).reshape(-1,1)))


outputs= sk_mse(p_sentiment,pred_sentiment1)

output1= r2_score(p_sentiment,pred_sentiment1)

print("Post MSE:", np.mean(outputs))
print("Post R2:", np.mean(output1))


C =  [0.89370571]
Headline MSE:  0.07238007411975393
Headline R2: -0.10588111146989698
C =  [0.94452278]
Post MSE: 0.03522617546012155
Post R2: -0.26055776983025947


In [429]:
ghosal_head = sparse.csc_matrix([p,a1,b1,c1])
ghosal_post = sparse.csc_matrix([p1,a2,b2,c2])


# ASPECT IN SENTIMENT

In [327]:
def define_model_sentiment(dropout,learning_rate,em,em_dim,input_shape,lstm_out, n_hidden_layer,em_trainable_flag,n_filters=100):
   
    input_context= Input(shape=(max_length,),name='Context')
    input_aspect =  Input(shape=(max_length,),name='Aspect')
    
    embedding1=Embedding(vocab_size, em_dim, weights = [eval(em)],input_length=max_length,trainable = False)
    aspect_embedding=Embedding(vocab_size ,em_dim , embeddings_initializer='uniform',trainable = True , input_length = max_length)(input_aspect)

    context= embedding1(input_context)    
    context=Dropout(0.2)(context)
    
    concat= concatenate([aspect_embedding,context])
    context=Dropout(0.2)(concat)

    c3=Conv1D(100,4,activation='relu')(context)    
    c3= LSTM(300,return_sequences = True)(c3)
    c3=LSTM(100)(c3)
    
    out=Dense(1,activation='sigmoid')(c3)

    model= Model(inputs=[input_context,input_aspect] ,outputs=out)
    

    optimizer = Adam(lr=learning_rate)

    model.compile(loss='mse', optimizer=optimizer, metrics=['cosine'])

    
    return model

In [328]:
model=define_model_sentiment(learning_rate=0.00069,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='embedding_matrix',
                     em_trainable_flag=False,
                     em_dim=300,
                     input_shape= X_aspect.shape[1]
                   )


In [329]:
EarlyStop= EarlyStopping(monitor='val_loss',patience=10,verbose=1)

model.fit(x=[trainX,train_aspect],y=train_data['sentiment'], epochs=100,batch_size=16,
             callbacks=[EarlyStop],validation_data=([head_X,head_aspect],head_data['sentiment']))

Train on 1173 samples, validate on 93 samples
Epoch 1/100
1173/1173 [==============================] - 24s 20ms/step - loss: 0.1724 - cosine_proximity: -0.3078 - val_loss: 0.2752 - val_cosine_proximity: -0.0538
Epoch 2/100
1173/1173 [==============================] - 17s 14ms/step - loss: 0.1599 - cosine_proximity: -0.3078 - val_loss: 0.2601 - val_cosine_proximity: -0.0538
Epoch 3/100
1173/1173 [==============================] - 15s 12ms/step - loss: 0.1586 - cosine_proximity: -0.3078 - val_loss: 0.2666 - val_cosine_proximity: -0.0538
Epoch 4/100
1173/1173 [==============================] - 17s 14ms/step - loss: 0.1530 - cosine_proximity: -0.3078 - val_loss: 0.2512 - val_cosine_proximity: -0.0538
Epoch 5/100
1173/1173 [==============================] - 18s 15ms/step - loss: 0.1508 - cosine_proximity: -0.3078 - val_loss: 0.2472 - val_cosine_proximity: -0.0538
Epoch 6/100
1173/1173 [==============================] - 17s 14ms/step - loss: 0.1503 - cosine_proximity: -0.3078 - val_loss: 0.2

In [330]:

pred_sentiment =model.predict([head_X,head_aspect])

outputs= sk_mse(h_sentiment,pred_sentiment)
output1= r2_score(h_sentiment,pred_sentiment)
print("C = ", cosine_similarity(np.array(h_sentiment).reshape(-1,1),np.array(pred_sentiment).reshape(-1,1)))
print("Headline MSE: ", np.mean(outputs))
print("Headline R2:", np.mean(output1))

pred_sentiment1 =model.predict([post_X,post_aspect])

#pred_sentiment = rescale(pred_sentiment,[0,1],[-1,1])

outputs= sk_mse(p_sentiment,pred_sentiment1)
output1= r2_score(p_sentiment,pred_sentiment1)
print("C = ", cosine_similarity(np.array(p_sentiment).reshape(-1,1),np.array(pred_sentiment1).reshape(-1,1)))
print("Post MSE:", np.mean(outputs))
print("Post R2:", np.mean(output1))


C =  [0.68950976]
Headline MSE:  0.22594306609084852
Headline R2: -2.519308900087049
C =  [0.76232906]
Post MSE: 0.1772782492869399
Post R2: -5.221213927521182


In [311]:
# 1 STACKED LSTM - GLOVE
pred_sentiment =model.predict([head_X,head_aspect])

outputs= sk_mse(h_sentiment,pred_sentiment)
output1= r2_score(h_sentiment,pred_sentiment)
print("C = ", cosine_similarity(np.array(h_sentiment).reshape(-1,1),np.array(pred_sentiment).reshape(-1,1)))
print("Headline MSE: ", np.mean(outputs))
print("Headline R2:", np.mean(output1))

pred_sentiment1 =model.predict([post_X,post_aspect])

#pred_sentiment = rescale(pred_sentiment,[0,1],[-1,1])

outputs= sk_mse(p_sentiment,pred_sentiment1)
output1= cp_sentiment,pred_sentiment1)
print("C = ", cosine_similarity(np.array(p_sentiment).reshape(-1,1),np.array(pred_sentiment1).reshape(-1,1)))
print("Post MSE:", np.mean(outputs))
print("Post R2:", np.mean(output1))

C =  [0.78494993]
Headline MSE:  0.1599008671054071
Headline R2: -1.275366972923393
C =  [0.88827377]
Post MSE: 0.08723961300822569
Post R2: -1.8681261816065158


# CLASSIFICATION

In [90]:
def convert_labels (trainY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,27), le.classes_, trainY

In [91]:
train_1hot,lable_encoding,train_y=convert_labels(train_data['aspect'])
data_1hot,lable_encoding,data_y=convert_labels(dataframe['aspect'])

#p_1hot,lable_encoding1,p_y=convert_labels(post['aspect'])

#new_train_1hot,_,new_train_y=convert_labels(new_train['aspect'])
head_1hot,_,head_y=convert_labels(head_data['aspect'])
post_1hot,_,post_y=convert_labels(post_data['aspect'])

In [92]:
train_y[0]

'Financial'

In [93]:
def get_class_from_pred(pred):
    return [lable_encoding[x.argmax()] for x in pred]

# OUR MODEL AND ATTENTION

In [435]:

def define_model_1(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,em_dim,max_length=16):
     
    input_context= Input(shape=(max_length,),name='Context')    
    
    embedding=Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag)
 
    context= embedding(input_context)
    
    context=Dropout(0.2)(context)
        
    a = Bidirectional(LSTM(300, return_sequences=True,recurrent_dropout=dropout))(context)
    
    a = Attention()(a)
    
    x=Dense(300,activation='relu')(a)
        
    out=Dense(27,activation='softmax')(x)

    model= Model(inputs=input_context ,outputs=out)
    
    optimizer = Adam(lr=learning_rate)
    
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    
    
   # print(model.summary())
    
    return model



In [438]:
Our_model = define_model_1(learning_rate=0.001,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='glove',
                     em_trainable_flag=False,
                     em_dim=200)


In [439]:
print("Our_model+attention")

EarlyStop= EarlyStopping(monitor='val_acc',patience=10,verbose=1)

Our_model.fit(x=trainX_1,y=train_1hot, epochs=100,batch_size=64,callbacks=[EarlyStop],validation_data=(post_X_1,post_1hot))


Our_model+attention
Train on 1173 samples, validate on 99 samples
Epoch 1/100
1173/1173 [==============================] - 15s 13ms/step - loss: 2.5731 - acc: 0.3487 - val_loss: 5.7051 - val_acc: 0.0000e+00
Epoch 2/100
1173/1173 [==============================] - 10s 8ms/step - loss: 2.0103 - acc: 0.4544 - val_loss: 6.0559 - val_acc: 0.0101
Epoch 3/100
1173/1173 [==============================] - 11s 9ms/step - loss: 1.6340 - acc: 0.5337 - val_loss: 6.3559 - val_acc: 0.0000e+00
Epoch 4/100
1173/1173 [==============================] - 9s 8ms/step - loss: 1.3125 - acc: 0.6232 - val_loss: 6.8383 - val_acc: 0.0000e+00
Epoch 5/100
1173/1173 [==============================] - 10s 8ms/step - loss: 1.0842 - acc: 0.6692 - val_loss: 7.6672 - val_acc: 0.0000e+00
Epoch 6/100
1173/1173 [==============================] - 10s 8ms/step - loss: 0.8960 - acc: 0.7425 - val_loss: 8.9355 - val_acc: 0.0000e+00
Epoch 7/100
1173/1173 [==============================] - 9s 8ms/step - loss: 0.7515 - acc: 0.7707 

In [440]:

print("Headlines")
pred8 =Our_model.predict(head_X_1)
pred= get_class_from_pred(pred8)
print("Accuracy: " ,accuracy_score(head_data['aspect'],pred))
print("Macro: ",f1_score(head_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(head_data['aspect'],pred,average='weighted'))


print()
pred1 =Our_model.predict(post_X_1)
print("Post")
pred= get_class_from_pred(pred1)
print("Accuracy: " ,accuracy_score(post_data['aspect'],pred))
print("Macro: ",f1_score(post_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(post_data['aspect'],pred,average='weighted'))


Headlines
Accuracy:  0.3010752688172043
Macro:  0.15251317856939414
Weighted:  0.2835542472315357



C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Post
Accuracy:  0.6666666666666666
Macro:  0.1852310988674625
Weighted:  0.7065316464306364


In [484]:
def train_model(model, feature_vector_train, label, feature_vector_valid,feature_vector_valid1):
    # fit the training dataset on the classifier
 
    model.fit(feature_vector_train, label)
        
    predictions = model.predict(feature_vector_valid)
    predictions1 = model.predict(feature_vector_valid1)
    predictions2 = model.predict(feature_vector_train)

    #pred=get_class_from_pred(predictions)
    #pred1=get_class_from_pred(predictions1)
    print("Train-> ")
    print("Accuracy: ",accuracy_score(predictions2,train_y))
    print("F1 score: ",f1_score(train_data['aspect'],predictions2,average='macro'))
    print("Weighted F1 score: ",f1_score(train_data['aspect'],predictions2,average='weighted'))
    print()
    
    
    print("Headlines-> ")
    print("Accuracy: ",accuracy_score(predictions,head_y))
    print("F1 score: ",f1_score(head_data['aspect'],predictions,average='macro'))
    print("Weighted F1 score: ",f1_score(head_data['aspect'],predictions,average='weighted'))
    print()
    
    print("Post-> ")
    print("Accuracy: ",accuracy_score(predictions1, post_y)) 
    print("F1 score: ",f1_score(post_data['aspect'],predictions1,average='macro'))
    print("Weighted F1 score: ",f1_score(post_data['aspect'],predictions1,average='weighted'))
     
    return predictions,predictions1,predictions2

In [443]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm


In [444]:
xtrain_count.shape

(1173, 3023)

In [445]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB


In [489]:
classifiers = [ 
                LogisticRegression(C = 100, random_state = 8),
                LogisticRegression(C = 1, random_state = 8),
                LogisticRegression(C = 0.01, random_state = 8),
                KNeighborsClassifier(3) ,
                SVC( kernel = 'rbf' , C = 0.025),
                DecisionTreeClassifier() ,
                RandomForestClassifier() , 
                AdaBoostClassifier(),
                GradientBoostingClassifier() 
              ]

TRAIN_PREDICTIONS=[]
HEAD_PREDICTIONS=[]
POST_PREDICTIONS=[]
for clf in classifiers:
    
    clf.fit( trainX , train_y)
    name = clf.__class__.__name__
    
    print( "="*30)
    print(name)
    
    print('******Results******')

    
    print("Headlines-> ")
    train_predictions = clf.predict(head_X)
    acc = accuracy_score( head_data['aspect'],train_predictions )
    print("Accuracy : {:.4%}".format(acc))
    print("F1 score: ",f1_score(head_data['aspect'],train_predictions,average='macro'))
    print("Weighted F1 score: ",f1_score(head_data['aspect'],train_predictions,average='weighted'))
    HEAD_PREDICTIONS.append(train_predictions)
    print()
    train_predictions = clf.predict(post_X)
    print("Post-> ")
    acc=accuracy_score(post_data['aspect'], train_predictions)
    print("Accuracy : {:.4%}".format(acc))
    print("F1 score: ",f1_score(post_data['aspect'],train_predictions,average='macro'))
    print("Weighted F1 score: ",f1_score(post_data['aspect'],train_predictions,average='weighted'))
    POST_PREDICTIONS.append(train_predictions)
    

LogisticRegression
******Results******
Headlines-> 
Accuracy : 11.8280%
F1 score:  0.010582010582010583
Weighted F1 score:  0.028673835125448032

Post-> 
Accuracy : 48.4848%
F1 score:  0.0909090909090909
Weighted F1 score:  0.404040404040404


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


LogisticRegression
******Results******
Headlines-> 
Accuracy : 11.8280%
F1 score:  0.01198257080610022
Weighted F1 score:  0.027830487033523088

Post-> 
Accuracy : 53.5354%
F1 score:  0.10028382213812678
Weighted F1 score:  0.38999264164827085
LogisticRegression
******Results******
Headlines-> 
Accuracy : 11.8280%
F1 score:  0.011111111111111112
Weighted F1 score:  0.028673835125448032

Post-> 
Accuracy : 54.5455%
F1 score:  0.1411764705882353
Weighted F1 score:  0.39215686274509803
KNeighborsClassifier
******Results******
Headlines-> 
Accuracy : 11.8280%
F1 score:  0.04412974122651542
Weighted F1 score:  0.09578403906187465

Post-> 
Accuracy : 23.2323%
F1 score:  0.031991153816823
Weighted F1 score:  0.27559742933799214
SVC
******Results******
Headlines-> 
Accuracy : 12.9032%
F1 score:  0.0126984126984127
Weighted F1 score:  0.029493087557603687

Post-> 
Accuracy : 55.5556%
F1 score:  0.17857142857142858
Weighted F1 score:  0.3968253968253968
DecisionTreeClassifier
******Results******

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


AdaBoostClassifier
******Results******
Headlines-> 
Accuracy : 10.7527%
F1 score:  0.010964912280701754
Weighted F1 score:  0.026881720430107527

Post-> 
Accuracy : 52.5253%
F1 score:  0.13959731543624163
Weighted F1 score:  0.38777032065622674


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


GradientBoostingClassifier
******Results******
Headlines-> 
Accuracy : 12.9032%
F1 score:  0.02643658810325477
Weighted F1 score:  0.060598819738604696

Post-> 
Accuracy : 37.3737%
F1 score:  0.050935940699720225
Weighted F1 score:  0.33323864819927806


### (1) Naive Bayes

In [485]:
#Naive Bayes on Count Vectors
HEAD_PREDICTIONS_2=[]
POST_PREDICTIONS_2=[]


print("NB, Count Vectors: ")
h_predictions,p_predictions,TRAIN=train_model(naive_bayes.MultinomialNB(), xtrain_count,train_y, head_count,post_count)
HEAD_PREDICTIONS_2.append(h_predictions)
POST_PREDICTIONS_2.append(p_predictions)

print()

# Naive Bayes on Word Level TF IDF Vectors
print ("NB, WordLevel TF-IDF: ")
h_predictions,p_predictions,TRAIN1=train_model(naive_bayes.MultinomialNB(), xtrain_tfidf,train_y,head_tfidf,post_tfidf)
print()
HEAD_PREDICTIONS_2.append(h_predictions)
POST_PREDICTIONS_2.append(p_predictions)

# Naive Bayes on Ngram Level TF IDF Vectors
print( "NB, N-Gram Vectors: ")
h_predictions,p_predictions,TRAIN2=train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram,train_y, head_tfidf_ngram,post_tfidf_ngram)
print()
HEAD_PREDICTIONS_2.append(h_predictions)
POST_PREDICTIONS_2.append(p_predictions)

# Naive Bayes on Character Level TF IDF Vectors
print ("NB, CharLevel Vectors: ")
h_predictions,p_predictions,TRAIN3=train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, head_tfidf_ngram_chars,post_tfidf_ngram_chars)
print()
HEAD_PREDICTIONS_2.append(h_predictions)
POST_PREDICTIONS_2.append(p_predictions)


print ("NB, Sending all the features ")
h_predictions,p_predictions,TRAIN4=train_model(naive_bayes.MultinomialNB(), X_aspect, train_y, x_test_1_aspect,x_test_2_aspect)
print()
HEAD_PREDICTIONS_2.append(h_predictions)
POST_PREDICTIONS_2.append(p_predictions)


NB, Count Vectors: 
Train-> 
Accuracy:  0.83461210571185
F1 score:  0.5398318964173976
Weighted F1 score:  0.8093349375443583

Headlines-> 
Accuracy:  0.20430107526881722
F1 score:  0.09240748931151407
Weighted F1 score:  0.11251468329267189

Post-> 
Accuracy:  0.7171717171717171
F1 score:  0.28548644338118023
Weighted F1 score:  0.6443381180223285

NB, WordLevel TF-IDF: 
Train-> 
Accuracy:  0.44501278772378516
F1 score:  0.08579416348426996
Weighted F1 score:  0.3225299686325696

Headlines-> 
Accuracy:  0.13978494623655913
F1 score:  0.0405982905982906
Weighted F1 score:  0.04590570719602977

Post-> 
Accuracy:  0.5555555555555556
F1 score:  0.17857142857142858
Weighted F1 score:  0.3968253968253968

NB, N-Gram Vectors: 
Train-> 
Accuracy:  0.38107416879795397
F1 score:  0.03004565865759332
Weighted F1 score:  0.21987978631758118

Headlines-> 
Accuracy:  0.12903225806451613
F1 score:  0.0126984126984127
Weighted F1 score:  0.029493087557603687

Post-> 
Accuracy:  0.5555555555555556
F1 

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train-> 
Accuracy:  0.6086956521739131
F1 score:  0.2861502360747593
Weighted F1 score:  0.5532417288131621

Headlines-> 
Accuracy:  0.13978494623655913
F1 score:  0.0405982905982906
Weighted F1 score:  0.04590570719602977

Post-> 
Accuracy:  0.5858585858585859
F1 score:  0.24461920529801326
Weighted F1 score:  0.4577396481369992



In [475]:
train_y.shape

(1173,)

# Vector Averaging

In [492]:
def create_model_architecture(input_size,input_size1):
    # create input layer 
    input_layer = Input((input_size, ), sparse=True)
    input_layer1 = Input((input_size1, ), sparse=True)
    
    # create hidden layer
    hidden_layer = Dense(500, activation="relu")(input_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    
    hidden_layer = Dense(50, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    hidden_layer = Dense(20, activation="relu")(hidden_layer)
    hidden_layer1 = Dropout(0.2)(hidden_layer)

     # create hidden layer
    hidden_layer = Dense(500, activation="relu")(input_layer1)
    hidden_layer = Dropout(0.2)(hidden_layer)
    
    hidden_layer = Dense(50, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)

    hidden_layer = Dense(20, activation="relu")(hidden_layer)
    hidden_layer2 = Dropout(0.2)(hidden_layer)

    concat=concatenate([hidden_layer1,hidden_layer2])
    output_layer = Dense(27, activation="softmax")(concat)
    output_layer = Dropout(0.2)(output_layer)
    


    classifier = Model(inputs = [input_layer,input_layer1], outputs = output_layer)
    classifier.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
    
    return classifier





In [493]:
model = create_model_architecture(X_s2v_g.shape[1],X_aspect.shape[1])

NameError: name 'Conv1d' is not defined

In [494]:
train_y.shape

(1173,)

In [465]:
model.fit(X_s2v_g, data_1hot,batch_size=64, epochs=30)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [<1173x200 sparse matrix of type '<class 'numpy.float64'>'
	with 234599 stored elements in Compressed Sparse Column format>]...

In [469]:

print("Headlines")
pred8 =model.predict([X_h2v_g,x_test_1_aspect])
pred= get_class_from_pred(pred8)
print("Accuracy: " ,accuracy_score(head_data['aspect'],pred))
print("Macro: ",f1_score(head_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(head_data['aspect'],pred,average='weighted'))


print()
pred1 =model.predict([X_p2v_g,x_test_2_aspect])
print("Post")
pred= get_class_from_pred(pred1)
print("Accuracy: " ,accuracy_score(post_data['aspect'],pred))
print("Macro: ",f1_score(post_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(post_data['aspect'],pred,average='weighted'))

#print("Headline accuracy: ", outputs)

Headlines
Accuracy:  0.11827956989247312
Macro:  0.011866235167206042
Weighted:  0.027560288130285

Post
Accuracy:  0.40404040404040403
Macro:  0.07299270072992702
Weighted:  0.32441200324412006


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# CLSTM

In [96]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args


dim_learning_rate = Categorical(categories=[0.01,0.1,0.001,0.0001],name='learning_rate')


dropout = Real(low=0.2, high=0.9,name='dropout')


para_batch_size = Categorical(categories=[16,32,64],name='batch_size')

dense_nodes = Integer(low=256, high=512, name='dense_nodes')


lstm_out = Integer(low=200, high=400, name='lstm_out')



para_filter_size = Categorical(categories=[3,4,5,6],name='filter_size')
para_n_filters = Categorical(categories=[100,200,300,400],name='n_filters')

#In[17]:


parameters = [dim_learning_rate,
              dropout,
              para_batch_size,
              lstm_out,
             para_filter_size,
              para_n_filters]





default_parameters = [0.001,0.2,16,200,4,100]

In [101]:

@use_named_args(dimensions=parameters)
def fitness(learning_rate,dropout,batch_size,lstm_out,filter_size,n_filters):
        

#        Print the hyper-parameters.
        print('----------------------------- combination ------------------')
        print('learning rate===> ',learning_rate)
        print('lstm_out:', lstm_out)
        print('dropout===>',dropout)
        print('batch_size===> ',batch_size)
        print('filter_size===> ',filter_size)
        print('num_filters===> ',n_filters)

        print()


        
        model = define_model(learning_rate=learning_rate,
                                 lstm_out=lstm_out,
                                 dropout=dropout,
                                 em='glove',
                                 em_trainable_flag=False,
                                 em_dim=200,
                                 n_filters=n_filters,
                                 filter_size=filter_size)

  
   
        earlystop = EarlyStopping(monitor='val_loss',patience=10,verbose=1, mode='auto')

        history = model.fit(x=trainX,
                            y=train_1hot,
                            epochs=30,
                            batch_size=batch_size,
                            callbacks=[earlystop],
                            validation_data=(head_X,head_1hot)
                            )
        accuracy = history.history['acc'][-1]
       
        print()
        print("Accuracy: {0:.2%}".format(accuracy))
        print()
        
        
        _,accuracy_1= model.evaluate(x=head_X , y=head_1hot)

        pred1 = model.predict(head_X)
        pred_class = get_class_from_pred(pred1)

        

        print("Test Headline")
        print("F1 Score Macro: ",f1_score(head_data['aspect'],pred_class,average='macro'))
        print("F1 Score Weighted: ",f1_score(head_data['aspect'],pred_class,average='weighted'))
        print("Accuracy: ",accuracy_1)
        print()

        _,accuracy_2 = model.evaluate(x=post_X , y=post_1hot)


        pred2= model.predict(post_X)
        pred_class = get_class_from_pred(pred2)

        print("Test Post")
        print("F1 Score Macro: ",f1_score(post_data['aspect'],pred_class,average='macro'))
        print("F1 Score Weighted: ",f1_score(post_data['aspect'],pred_class,average='weighted'))
        print("Accuracy: ",accuracy_2)
        print()


       

        K.clear_session()
        
        print("Session cleared ")
        
        return -accuracy
        



In [ ]:
seed=7
np.random.seed(seed)


#fitness(default_parameters)


search_result = gp_minimize(func=fitness,
                            dimensions=parameters,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters)

----------------------------- combination ------------------
learning rate===>  0.001
lstm_out: 200
dropout===> 0.2
batch_size===>  16
filter_size===>  4
num_filters===>  100

Train on 1173 samples, validate on 93 samples
Epoch 1/30
1173/1173 [==============================] - 4s 4ms/step - loss: 2.5756 - acc: 0.3674 - val_loss: 4.1007 - val_acc: 0.0108
Epoch 2/30
1173/1173 [==============================] - 3s 2ms/step - loss: 2.4820 - acc: 0.3725 - val_loss: 3.9696 - val_acc: 0.0108
Epoch 3/30
1173/1173 [==============================] - 3s 2ms/step - loss: 2.4735 - acc: 0.3725 - val_loss: 3.9040 - val_acc: 0.0108
Epoch 4/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4766 - acc: 0.3717 - val_loss: 4.1336 - val_acc: 0.0108
Epoch 5/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4724 - acc: 0.3725 - val_loss: 3.9995 - val_acc: 0.0108
Epoch 6/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4668 - acc: 0.3725 - val_loss:

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 1173 samples, validate on 93 samples
Epoch 1/30
1173/1173 [==============================] - 4s 4ms/step - loss: 2.6237 - acc: 0.3640 - val_loss: 3.9844 - val_acc: 0.0108
Epoch 2/30
1173/1173 [==============================] - 4s 3ms/step - loss: 2.4675 - acc: 0.3725 - val_loss: 3.8952 - val_acc: 0.0108
Epoch 3/30
1173/1173 [==============================] - 4s 3ms/step - loss: 2.4713 - acc: 0.3725 - val_loss: 4.0136 - val_acc: 0.0108
Epoch 4/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4690 - acc: 0.3725 - val_loss: 4.0290 - val_acc: 0.0108
Epoch 5/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4643 - acc: 0.3725 - val_loss: 3.8601 - val_acc: 0.0108
Epoch 6/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4589 - acc: 0.3725 - val_loss: 4.0494 - val_acc: 0.0108 loss: 2.4
Epoch 7/30
1173/1173 [==============================] - 3s 3ms/step - loss: 2.4523 - acc: 0.3725 - val_loss: 4.0285 - val_acc: 0.0108
Epoch 

In [99]:
max_length = 16

In [95]:
def define_model(dropout,learning_rate,em,em_dim,lstm_out,em_trainable_flag,n_filters=100,filter_size=4):
   
    input_context= Input(shape=(max_length,),name='Context')
   
    embedding1=Embedding(vocab_size, em_dim, weights = [eval(em)],input_length=max_length,trainable = False)
    
    context= embedding1(input_context)    
    context=Dropout(dropout)(context)
    
    c3=Conv1D(n_filters,filter_size,activation='relu')(context)    
    c3= LSTM(lstm_out,return_sequences = True)(c3)
    c3=LSTM(lstm_out)(c3)
    
    out=Dense(27,activation='softmax')(c3)

    model= Model(inputs=input_context ,outputs=out)
    

    optimizer = Adam(lr=learning_rate)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    
    return model

In [ ]:
Our_model = define_model(learning_rate=0.001,
                     dropout=0.5,
                     lstm_out=300,
                     n_hidden_layer=1,
                     em='glove',
                     em_trainable_flag=False,
                     em_dim=200)


In [502]:
EarlyStop= EarlyStopping(monitor='val_acc',patience=10,verbose=1)

Our_model.fit(x=trainX_1,y=train_1hot, epochs=100,batch_size=64,callbacks=[EarlyStop],validation_data=(post_X_1,post_1hot))


Train on 1173 samples, validate on 99 samples
Epoch 1/100
1173/1173 [==============================] - 5s 5ms/step - loss: 2.7465 - acc: 0.3495 - val_loss: 4.3676 - val_acc: 0.0000e+00
Epoch 2/100
1173/1173 [==============================] - 1s 922us/step - loss: 2.3933 - acc: 0.3725 - val_loss: 4.6635 - val_acc: 0.0000e+00
Epoch 3/100
1173/1173 [==============================] - 1s 917us/step - loss: 2.1443 - acc: 0.3990 - val_loss: 5.1352 - val_acc: 0.0000e+00
Epoch 4/100
1173/1173 [==============================] - 1s 915us/step - loss: 2.0209 - acc: 0.4169 - val_loss: 5.3956 - val_acc: 0.0000e+00
Epoch 5/100
1173/1173 [==============================] - 1s 908us/step - loss: 1.9269 - acc: 0.4510 - val_loss: 5.9812 - val_acc: 0.0000e+00
Epoch 6/100
1173/1173 [==============================] - 1s 928us/step - loss: 1.8416 - acc: 0.4689 - val_loss: 6.6469 - val_acc: 0.0000e+00
Epoch 7/100
1173/1173 [==============================] - 1s 911us/step - loss: 1.6892 - acc: 0.5047 - val_loss

In [503]:

print("Headlines")
pred8 =Our_model.predict(head_X_1)
pred= get_class_from_pred(pred8)
print("Accuracy: " ,accuracy_score(head_data['aspect'],pred))
print("Macro: ",f1_score(head_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(head_data['aspect'],pred,average='weighted'))


print()
pred1 =Our_model.predict(post_X_1)
print("Post")
pred= get_class_from_pred(pred1)
print("Accuracy: " ,accuracy_score(post_data['aspect'],pred))
print("Macro: ",f1_score(post_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(post_data['aspect'],pred,average='weighted'))


Headlines
Accuracy:  0.1935483870967742
Macro:  0.11756945908546576
Weighted:  0.16769974130098622

Post
Accuracy:  0.6161616161616161
Macro:  0.1373684567859325
Weighted:  0.6435288453823317


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Tfidf Features

In [94]:

print("Headlines")
pred2 =model.predict(x_test_1_aspect)
pred= get_class_from_pred(pred2)
print("Accuracy: " ,accuracy_score(head_data['aspect'],pred))
print("Macro: ",f1_score(head_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(head_data['aspect'],pred,average='weighted'))


print()
pred3 =model.predict(x_test_2_aspect)
print("Post")
pred= get_class_from_pred(pred3)
print("Accuracy: " ,accuracy_score(post_data['aspect'],pred))
print("Macro: ",f1_score(post_data['aspect'],pred,average='macro'))
print("Weighted: ",f1_score(post_data['aspect'],pred,average='weighted'))

#print("Headline accuracy: ", outputs)

Headlines


NameError: name 'model' is not defined